In [1]:
import sqlite3
import pandas as pd
from datetime import datetime
from pprint import pprint

from functions.dataLoading import getPlayersPastPVE, getPGCR
from functions.database import insertActivity, insertInstanceDetails, insertCharacter, updatedPlayer

In [2]:
con = sqlite3.connect('userdb.sqlite3')
cur = con.cursor()

In [3]:
playerlist = cur.execute('SELECT destinyID FROM discordGuardiansToken')
playerlist = [p[0] for p in playerlist]
len(playerlist)

57

In [4]:
for player in playerlist:
    pastpve = getPlayersPastPVE(player)
    print(pastpve[0])
    break

{'period': '2020-01-27T11:42:14Z', 'activityDetails': {'referenceId': 4166562681, 'directorActivityHash': 4166562681, 'instanceId': '5655310385', 'mode': 6, 'modes': [7, 6], 'isPrivate': False, 'membershipType': 3}, 'values': {'assists': {'statId': 'assists', 'basic': {'value': 2.0, 'displayValue': '2'}}, 'completed': {'statId': 'completed', 'basic': {'value': 0.0, 'displayValue': 'No'}}, 'deaths': {'statId': 'deaths', 'basic': {'value': 0.0, 'displayValue': '0'}}, 'kills': {'statId': 'kills', 'basic': {'value': 42.0, 'displayValue': '42'}}, 'opponentsDefeated': {'statId': 'opponentsDefeated', 'basic': {'value': 44.0, 'displayValue': '44'}}, 'efficiency': {'statId': 'efficiency', 'basic': {'value': 44.0, 'displayValue': '44.00'}}, 'killsDeathsRatio': {'statId': 'killsDeathsRatio', 'basic': {'value': 42.0, 'displayValue': '42.00'}}, 'killsDeathsAssists': {'statId': 'killsDeathsAssists', 'basic': {'value': 43.0, 'displayValue': '43.00'}}, 'score': {'statId': 'score', 'basic': {'value': 0

In [5]:
for pve in pastpve:
    period = datetime.strptime(pve['period'], "%Y-%m-%dT%H:%M:%SZ")
    activityHash = pve['activityDetails']['directorActivityHash']
    instanceID = pve['activityDetails']['instanceId']
    timePlayedSeconds = int(pve['values']['timePlayedSeconds']['basic']['value'])
    completed = int(pve['values']['completed']['basic']['value'])
    if completed and not int(pve['values']['completionReason']['basic']['value']):
        pgcrdata = getPGCR(instanceID)['Response']
        startingPhaseIndex = pgcrdata['startingPhaseIndex']
        deaths = 0
        playercount = 0
        for player in pgcrdata['entries']:
            lightlevel = player['player']['lightLevel']
            characterid = player['characterId']
            playerdeaths = int(player['values']['deaths']['basic']['displayValue'])
            deaths += playerdeaths
            displayname = player['player']['destinyUserInfo']['displayName']
            completed = int(player['values']['completed']['basic']['value'])
            playercount += completed
            opponentsDefeated = player['values']['opponentsDefeated']['basic']['value']
            system = player['player']['destinyUserInfo']['membershipType']
            insertCharacter(destinyID, characterid, system)
            insertInstanceDetails(instanceID, playerID, characterID, lightlevel, displayname, deaths, opponentsDefeated, completed)
        insertActivity(instanceID, activityHash, timePlayedSeconds, period, startingPhaseIndex, deaths, playercount)

{'characterId': '2305843009300408732',
 'extended': {'values': {'precisionKills': {'basic': {'displayValue': '8',
                                                      'value': 8.0}},
                         'weaponKillsAbility': {'basic': {'displayValue': '0',
                                                          'value': 0.0}},
                         'weaponKillsGrenade': {'basic': {'displayValue': '2',
                                                          'value': 2.0}},
                         'weaponKillsMelee': {'basic': {'displayValue': '9',
                                                        'value': 9.0}},
                         'weaponKillsSuper': {'basic': {'displayValue': '7',
                                                        'value': 7.0}}},
              'weapons': [{'referenceId': 2907129557,
                           'values': {'uniqueWeaponKills': {'basic': {'displayValue': '24',
                                                                 